In [13]:
import pathlib

In [8]:
from fastai import *
from fastai.text import *
from fastai.docs import *

In [9]:
untar_data(IMDB_PATH)
IMDB_PATH

PosixPath('../data/imdb_sample')

In [11]:
df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()

,0,1
0,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,1,This is a extremely well-made film. The acting...
2,0,Every once in a long while a movie will come a...
3,1,Name just says it all. I watched this movie wi...
4,0,This movie succeeds at being one of the most u...


In [15]:
cat ../data/imdb_sample/classes.txt

negative
positive


In [16]:
classes = read_classes(IMDB_PATH/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

In [17]:
train_ds = TextDataset.from_csv(IMDB_PATH, name='train', classes=classes)

Tokenizing train.


Numericalizing train.


In [25]:
type(train_ds)

fastai.text.data.TextDataset

In [26]:
type(data_lm)

fastai.data.DataBunch

In [24]:
data_lm = text_data_from_csv(Path(IMDB_PATH), data_func=lm_data)

Tokenizing valid.


Numericalizing valid.


In [28]:
# Classifier model data
data_clas = text_data_from_csv(Path(IMDB_PATH), data_func=classifier_data, vocab=data_lm.train_ds.vocab)

In [33]:
data_lm??

In [29]:
download_wt103_model()

In [30]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

Total time: 00:04
epoch  train loss  valid loss  accuracy
0      4.688786    4.201121    0.246745  (00:04)



In [31]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

Total time: 00:27
epoch  train loss  valid loss  accuracy
0      4.465899    4.120743    0.255161  (00:05)
1      4.350593    4.043670    0.261218  (00:05)
2      4.228394    4.005251    0.264912  (00:05)
3      4.142157    3.991989    0.264953  (00:05)
4      4.076058    3.988415    0.265019  (00:05)



In [32]:
learn.save_encoder('ft_enc')

In [35]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

Total time: 00:04
epoch  train loss  valid loss  accuracy
0      0.617495    0.633280    0.710000  (00:04)



In [36]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.532020    0.534861    0.760000  (00:05)



In [37]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

Total time: 00:08
epoch  train loss  valid loss  accuracy
0      0.430037    0.511754    0.765000  (00:08)

